In [ ]:
from __future__ import division

import os
import urllib, cStringIO

import pymongo as pm

import matplotlib
from matplotlib import pylab, mlab, pyplot
%matplotlib inline
from IPython.core.pylabtools import figsize, getfigs
plt = pyplot
import seaborn as sns
sns.set_context('poster')
sns.set_style('white')

import numpy as np
import scipy.stats as stats
import pandas as pd
import json
import re

from PIL import Image
import base64
import sys

from svgpathtools import parse_path
from IPython.display import clear_output

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")

### Setup

In [ ]:
# directory & file hierarchy
proj_dir = os.path.abspath('../..')
analysis_dir = os.getcwd()
results_dir = os.path.join(proj_dir,'results')
plot_dir = os.path.join(results_dir,'plots')
csv_dir = os.path.join(results_dir,'csv')
exp_dir = os.path.abspath(os.path.join(proj_dir,'experiments'))
sketch_dir = os.path.abspath(os.path.join(proj_dir,'sketches'))

## add helpers to python path
if os.path.join(proj_dir,'analysis') not in sys.path:
    sys.path.append(os.path.join(proj_dir,'analysis'))

if not os.path.exists(results_dir):
    os.makedirs(results_dir)  

if not os.path.exists(plot_dir):
    os.makedirs(plot_dir)   
    
if not os.path.exists(csv_dir):
    os.makedirs(csv_dir)  
    
## add helpers to python path
if os.path.join(proj_dir,'analysis') not in sys.path:
    sys.path.append(os.path.join(proj_dir,'analysis'))        
    
# Assign variables within imported analysis helpers
import analysis_helpers as h
if sys.version_info[0]>=3:
    from importlib import reload
reload(h)

#### set up connection to mongo

In [ ]:
#### set vars 
import pandas as pd
auth = pd.read_csv('auth.txt', header = None) # this auth.txt file contains the password for the sketchloop user
pswd = auth.values[0][0]
user = 'sketchloop'
host = 'rxdhawkins.me' ## cocolab ip address

# have to fix this to be able to analyze from local
import pymongo as pm
conn = pm.MongoClient('mongodb://sketchloop:' + pswd + '@127.0.0.1')
db = conn['semantic_parts']
coll = db['sketchpad_basic']

# which iteration name should we use?
iterationName = 'pilot0'

### get some descriptive stats on our annotation data!

In [ ]:
num_annotations = coll.find({'iterationName':iterationName}).count()
print 'We have {} annotations so far.'.format(num_annotations)

In [ ]:
## get list of valid assignments
unique_assignments = coll.find({'iterationName':iterationName}).distinct('aID')

sketch_ids = []
## construct list of unique sketches
for this_aID in unique_assignments:
    recs = coll.find({'$and': [{'iterationName':iterationName}, {'aID':this_aID}]}).sort('time')
    for rec in recs:
        print rec['originalGameID'], rec['originalTrialNum']
        clear_output(wait=True)
        sketch_ids.append('{}_{}'.format(rec['originalGameID'],str(rec['originalTrialNum']).zfill(2)))
    
print 'Finished constructing list of sketch IDs.'

## visualize the number of sketches that have been annotated k times
from collections import Counter
h = plt.hist(Counter(sketch_ids).values())
sns.set_context('talk')
sns.set_style('whitegrid')
plt.xlim(0,4.5)
plt.ylim(0,1200)
t = plt.xticks(np.arange(0,5,1))
t = plt.ylabel('num sketches')
t = plt.xlabel('num times annotated')

In [ ]:
## check score for particular worker
w = ''
coll.find_one({ '$and': [{'iterationName':iterationName}, \
                        {'wID': w}]}, \
                        sort=[("bonus", -1)])["bonus"]

#### build up annotation data csv

In [ ]:
## TODO remember to ignore researcher worker ID's in our dataset
jefan = ['A1MMCS8S8CTWKU','A1MMCS8S8CTWKV','A1MMCS8S8CTWKS']
hawkrobe = ['A1BOIDKD33QSDK']
megsano = ['A1DVQQLVZR7W6I']
kmukherjee = ['A1WU4IHJNQGVAY']
researchers = jefan + hawkrobe + megsano + kmukherjee

In [ ]:
## TODO use a helper function to get complete and valid HITs 

In [ ]:
## categories that are in play
main_category = 'chair'
dataset = 'chairs4'

In [ ]:
## get list of unique_assignments
unique_assignments = coll.find({'iterationName':iterationName}).distinct('aID')

### initialize a bunch of stuff
orig_gameID = [] # the gameID from which this sketch was sourced
orig_trial_num = [] # the trialnum in the original game from which this sketch was sourced -- 
sketch_id = [] # concatenation of orig_gameID and orig_trial_num -- 
assignmentID = [] # the session in which this annotation was collected -- 
annotation_id = [] # the unique ID for each annotation trial (different for each session the same sketch appears in)
category = [] # e.g., "chair"
target = [] # e.g., "inlay"
condition = [] # e.g., "closer" vs. "further" or "repeated" vs. "control
trial_num = []
time_submitted = [] # when the participant clicked "next sketch"
time_labeled = [] # unique to each spline labeled
time_clicked = [] # when this spline was clicked/selected
num_strokes_in_sketch = [] # how many strokes in this sketch
num_splines_in_sketch = [] # how many spline elements in this sketch
stroke_num = [] # which stroke number this labeled spline came from
cumulative_spline_num = [] # spline index in the cumulative spline sequence for the entire sketch
within_stroke_spline_num = [] # spline index for the current stroke
label = [] # the label provided by the participant
spline_svg_string = [] # the svg spline string that earned this label
sketch_svg_string = [] # the entire svg string correponding to this sketch
annotation_flag = [] # this is True if all splines were labeled as the same thing

## loop through all the unique assignments that have submitted things
for this_assignment, aID in enumerate(unique_assignments):
    if this_assignment%50==0:
        print 'Analyzing sketches from assignment {} of {}  ...'.format(this_assignment, len(unique_assignments))

    ### get all the sketch recs for this assignment
    sketch_recs = coll.find({'$and': [{'iterationName':iterationName}, {'aID':aID}]}).sort('time')

    try:

        for sketch_ind,sketch in enumerate(sketch_recs):
            ## get annotations embedded within record
            annotations_string = sketch['annotations']  
            ## convert to json dictionary
            _annotations_dict = json.loads(annotations_string)   
            annotations_dict = _annotations_dict[0][main_category]
            num_splines = len(annotations_dict)
            for annotation in annotations_dict:
                assert sketch['numSplines']==num_splines                
                ## get spline-level metadata
                label.append(annotation['label'])
                stroke_num.append(annotation['strokeNum'])
                spline_svg_string.append(annotation['svgString'])
                cumulative_spline_num.append(annotation['cumulativeSplineNum'])
                within_stroke_spline_num.append(annotation['withinStrokeSplineNum'])
                time_clicked.append(annotation['timeClicked'])
                time_labeled.append(annotation['timeLabeled'])
                ## get sketch-level metadata
                orig_gameID.append(sketch['originalGameID'])   
                orig_trial_num.append(sketch['originalTrialNum'])
                sketch_id.append('{}_{}'.format(sketch['originalGameID'],sketch['originalTrialNum']))
                annotation_id.append('{}_{}_{}'.format(sketch['originalGameID'],sketch['originalTrialNum'],sketch['aID']))
                assignmentID.append(sketch['aID'])
                category.append(sketch['category'])
                target.append(sketch['target'])
                condition.append(sketch['condition'])
                time_submitted.append(sketch['time'])
                trial_num.append(sketch['trialNum'])
                num_splines_in_sketch.append(sketch['numSplines'])
                num_strokes_in_sketch.append(sketch['numStrokes'])
                sketch_svg_string.append(sketch['svg'])
                annotation_flag.append(sketch['sameAnnotflag'])
                
    except AssertionError:
        print 'There were unequal numbers for sketch["numSplines"] vs. num_splines for sketch {} from {}'.\
                format(sketch['trialNum'], sketch['aID'])


In [ ]:
## make group data csv 
D = pd.DataFrame([orig_gameID, orig_trial_num, sketch_id, category, assignmentID,  target, \
                  annotation_id, condition, trial_num, time_submitted,\
                 time_labeled, time_clicked, num_strokes_in_sketch, num_splines_in_sketch,\
                 stroke_num, cumulative_spline_num, within_stroke_spline_num, label,\
                 spline_svg_string, sketch_svg_string])
D = D.transpose()
D.columns = ['orig_gameID', 'orig_trial_num', 'sketch_id', 'category', 'assignmentID', 'target',\
             'annotation_id', 'condition', 'trial_num', 'time_submitted',\
             'time_labeled', 'time_clicked', 'num_strokes_in_sketch', 'num_splines_in_sketch',\
             'stroke_num', 'cumulative_spline_num', 'within_stroke_spline_num', 'label',\
             'spline_svg_string', 'sketch_svg_string']
D=D[D['assignmentID']!='']
print 'Annotations dataframe contains {} rows and {} columns.'.format(D.shape[0],D.shape[1])

In [ ]:
## save out csv to results dir
D.to_csv(os.path.join(results_dir,'svg_annotations_{}.csv'.format(dataset)))

#### get descriptive stats on annotation data

In [ ]:
## Some starting questions about sketchpad_basic dataset

## Annotation task
## How consistent are annotators for the same object? Inter-rater reliability

## Content
## What does the part distribution look like for each object? Presence/absence.
## How does the part distribution differ between contexts, matched by object?

## Contenty Style
## Here we might use arc length as a measure of how much was expended for each part.
## How much arc length is spent on each part 
## Is the entropy of the part distribution (measuring arc length) different for close trials than far trials?

## Dynamics
## Are people drawing each part "in succession?" (Do people plan & execute their drawings in terms of parts?)
## Are people starting out producing longer strokes, than shorter strokes? Look at time series within 
## sketch for the arc lengths of strokes produced .... 

### Make visualizations of dataset

##### How many sketches do we have annotated?

In [ ]:
unique_sketches = np.unique(D['sketch_id'].values)
print 'We have {} unique sketches.'.format(len(unique_sketches))

#### Number of times each sketch has been annotated

In [ ]:
## get number of times it has been annotated
num_times_annotated = []
for this_sketch_id in unique_sketches:
    num_times_annotated.append(D[D['sketch_id']==this_sketch_id]['assignmentID'].nunique())
    
## make a histogram
sns.set_context('talk')
plt.figure(figsize=(4,4))
h = plt.hist(num_times_annotated)
plt.title('# times each sketch has been annotated')
plt.ylabel('number of sketches')

##### check to make sure that for each sketch, you have a unique number of splines associated with it

In [ ]:
unique_sketches = np.unique(D['sketch_id'].values)
for this_sketch in unique_sketches:
    assert len(np.unique(D[D['sketch_id']==this_sketch]['num_splines_in_sketch'].values))==1

##### How long do sketches take to annotate? Do more complex sketches (i.e., those consisting of more splines) take longer to annotate? [Sanity check]

In [ ]:
#### first, we will get the amount of time taken to annotate each sketch
unique_annotation_trials = np.unique(D['annotation_id'].values)

In [ ]:
## get annotation time for each annotation trial
annotation_time = []
spline_number_in_sketch = []

for this_annotation_trial in unique_annotation_trials:
    earliest_click = float(np.min(D[D['annotation_id']==this_annotation_trial]['time_clicked']))
    ## all of the splines were submitted at the same time, so time_submitted should be identical for all splines in an annotation trial
    assert len(np.unique(D[D['annotation_id']==this_annotation_trial]['time_submitted'].values))==1
    final_submission = np.unique(D[D['annotation_id']==this_annotation_trial]['time_submitted'].values)[0]
    annotation_time.append(final_submission-earliest_click)        
    if np.isnan(final_submission) or np.isnan(earliest_click):
        print 'One of these timestamps is a NaN. Probably means that the participant skipped this trial:'
        print this_annotation_trial
        print 'final_submission: {}, earliest_click: {}'.format(final_submission, earliest_click)
    #### then we will extract how "complex" each sketch i
    assert len(np.unique(D[D['annotation_id']==this_annotation_trial]['num_splines_in_sketch'].values))==1    
    spline_number_in_sketch.append(np.unique(D[D['annotation_id']==this_annotation_trial]['num_splines_in_sketch'])[0])
    

In [ ]:
## convert annotation time to seconds
annotation_time_seconds = np.array(annotation_time)/1000

## make dataframe with annotation time and spline number
unique_annotation_trials, spline_number_in_sketch, annotation_time_seconds = map(list, [unique_annotation_trials, spline_number_in_sketch, annotation_time_seconds])
T = pd.DataFrame([unique_annotation_trials,spline_number_in_sketch,annotation_time_seconds])
T = T.transpose()
T.columns = ['annotation_trial','spline_number_in_sketch','annotation_time']

## some preprocessing of T
reload(h)

## make numeric types
T = h.convert_numeric(T,'spline_number_in_sketch')
T = h.convert_numeric(T,'annotation_time')

## also remove the skipped trial where annotation time is a NaN
T = T[~np.isnan(T['annotation_time'])]

In [ ]:
## make scatterplot of relationship between annotation time and spline number
sns.scatterplot(x='annotation_time',
                y='spline_number_in_sketch',
                data=T)
plt.ylabel('spline number in sketch')
plt.xlabel('annotation time (s)')
plt.title('do more complex sketches take longer to annotate?')
plt.xlim(0,60*5) ## 5 minute cutoff ...

In [ ]:
r, p = stats.spearmanr(T['annotation_time'],T['spline_number_in_sketch'])
print 'Spearman correlation between annotation time and spline number in sketch' 
print 'r = {}, p = {}'.format(r,p)

##### Make histogram of part word occurrence distribution

In [ ]:
## get the list of unique labels applied to chairs
unique_labels = np.unique(D.label.values)

## some deborkification of the unique label list
unique_labels = [i for i in unique_labels if i is not None]
unique_labels = [i for i in unique_labels if len(i)<900]

proper_labels = []

In [ ]:
print 'After some light deborkificiation, there are {} unique part labels in our dataset.'.format(len(unique_labels))

In [ ]:
## create a dataframe that is [num_sketches x num_labels] where each cell is incremented by 1
## when that part word appears as an annotation to that sketch
unique_labels = np.array(unique_labels)

In [ ]:
unique_labels

In [ ]:
## initialize matrix that has the correct dimensions
Label_Vec = np.zeros((len(unique_sketches),len(unique_labels)), dtype=int)

for s,this_sketch in enumerate(unique_sketches):
    label_vec = np.zeros(len(unique_labels),dtype=int)
    DS = D[D['sketch_id']==this_sketch]
    annotation_ids = np.unique(DS['annotation_id'].values)    
    for this_annotation in annotation_ids:
        DSA = DS[(DS['sketch_id']==this_sketch) & (DS['annotation_id']==this_annotation)]
        label_list = np.unique(DSA.label.values)
        for this_label in label_list:
            label_ind = unique_labels==this_label
            label_vec[label_ind] += 1
    Label_Vec[s,:]=label_vec                    

In [ ]:
thresh = 100
print 'These are the labels that appear at least {} times:'.format(thresh)
print unique_labels[np.sum(Label_Vec,0)>thresh]

#### read in stroke dataframe in order to get part distribution

In [ ]:
strokes = pd.read_csv(os.path.join(results_dir,'stroke_df_lite.csv'))

In [ ]:
strokes.drop(columns=['Unnamed: 0'], inplace=True)

In [ ]:
A = strokes.groupby(['condition','sketch_id'])['stroke_num'].count()

In [ ]:
B = A.reset_index()

In [ ]:
array = np.array([['a',1],['b',2]])

In [ ]:
where= np.where(array=='b')

In [ ]:
int(array[where[0],where[1]+1].item())+1

In [ ]:
where